In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv


In [2]:
import tensorflow as tf

from tqdm.notebook import tqdm, trange

In [3]:
data_dir = '/kaggle/input/competitive-data-science-predict-future-sales'

In [4]:
df_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_shops = pd.read_csv(os.path.join(data_dir, 'shops.csv'))
df_item_categories = pd.read_csv(os.path.join(data_dir, 'item_categories.csv'))
df_items = pd.read_csv(os.path.join(data_dir, 'items.csv'))
df_train = pd.read_csv(os.path.join(data_dir, 'sales_train.csv'))

In [5]:
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [6]:
df_train['date'] = pd.to_datetime(df_train['date'], format='%d.%m.%Y')
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [7]:
df_train[(df_train['shop_id'] == 0) & (df_train['item_id'] == 30)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
173344,2013-02-21,1,0,30,265.0,2.0
173345,2013-02-20,1,0,30,265.0,2.0
173346,2013-02-18,1,0,30,265.0,4.0
173347,2013-02-17,1,0,30,265.0,4.0
173348,2013-02-16,1,0,30,265.0,9.0
173349,2013-02-15,1,0,30,265.0,2.0
173361,2013-02-22,1,0,30,265.0,2.0
173402,2013-02-23,1,0,30,265.0,3.0
173441,2013-02-26,1,0,30,265.0,3.0


In [8]:
train_keys = df_train[['shop_id', 'item_id']].values
train_keys = [tuple(x) for x in train_keys]
len(train_keys)

2935849

In [9]:
train_keys = dict.fromkeys(train_keys)
len(train_keys)

424124

In [10]:
test_keys = df_test[['shop_id', 'item_id']].values
test_keys = [tuple(x) for x in test_keys]
len(test_keys)

214200

In [11]:
test_keys = dict.fromkeys(test_keys)
len(test_keys)

214200

In [12]:
valid, in_valid = 0, 0
for key in test_keys:
    if key in train_keys:
        valid += 1
    else:
        in_valid += 1
valid, in_valid

(111404, 102796)

In [13]:
len(train_keys) + in_valid

526920

In [14]:
all_keys = list(dict.fromkeys(list(train_keys) + list(test_keys)))
len(all_keys)

526920

In [15]:
df_group = df_train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price': 'mean', 'item_cnt_day': 'sum'})
df_group

item_price  item_cnt_day
date_block_num shop_id item_id                          
0              0       32            221.0           6.0
                       33            347.0           3.0
                       35            247.0           1.0
                       43            221.0           1.0
                       51            128.5           2.0
...                                    ...           ...
33             59      22087         119.0           6.0
                       22088         119.0           2.0
                       22091         179.0           1.0
                       22100         629.0           1.0
                       22102        1250.0           1.0

[1609124 rows x 2 columns]

In [16]:
df_group = df_group.reset_index()
df_group

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,0,32,221.0,6.0
1,0,0,33,347.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,128.5,2.0
...,...,...,...,...,...
1609119,33,59,22087,119.0,6.0
1609120,33,59,22088,119.0,2.0
1609121,33,59,22091,179.0,1.0
1609122,33,59,22100,629.0,1.0


In [17]:
df_group[(df_group['shop_id'] == 0) & (df_group['item_id'] == 30)]

,date_block_num,shop_id,item_id,item_price,item_cnt_day
63224,1,0,30,265.0,31.0


In [18]:
df_pivot_cnt = df_group.pivot_table(index = ['shop_id', 'item_id'], values = ['item_cnt_day'], columns = ['date_block_num'], fill_value = 0, aggfunc='sum')
df_pivot_cnt = df_pivot_cnt.reset_index()
df_pivot_cnt

shop_id item_id item_cnt_day                        ...        \
date_block_num                            0   1  2  3  4  5  6  7  ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0  ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0  ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0  ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0  ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0  ...  0  0   
...                ...     ...          ...  .. .. .. .. .. .. ..  ... .. ..   
424119              59   22154            1   0  0  0  0  0  0  0  ...  0  0   
424120              59   22155            0   0  0  0  0  0  1  0  ...  0  0   
424121              59   22162            0   0  0  0  0  0  0  0  ...  0  9   
424122              59   22164            0   0  0  0  0  0  0  0  ...  0  2   
424123              59   22167            0   0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
424119          0  0  0  0  0  0  0  0  
424120          0  0  0  0  0  0  0  0  
424121          4  1  1  0  0  1  0  0  
424122          1  2  0  0  1  0  0  0  
424123          0  0  0  0  0  0  0  0  

[424124 rows x 36 columns]

In [19]:
df_merge_cnt = pd.merge(df_test, df_pivot_cnt, on = ['item_id','shop_id'], how = 'left')
df_merge_cnt

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_merge_cnt = df_merge_cnt.fillna(0)
df_merge_cnt

,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
train_ids = df_merge_cnt[['shop_id', 'shop_id']].values
train_ids

array([[ 5,  5],
       [ 5,  5],
       [ 5,  5],
       ...,
       [45, 45],
       [45, 45],
       [45, 45]])

In [22]:
test_ids = df_test[['shop_id', 'shop_id']].values
test_ids

array([[ 5,  5],
       [ 5,  5],
       [ 5,  5],
       ...,
       [45, 45],
       [45, 45],
       [45, 45]])

In [23]:
np.array_equal(train_ids, test_ids)

True

In [24]:
cnt_matrix = df_merge_cnt.drop(['ID', 'shop_id', 'item_id'], axis=1).values
cnt_matrix = np.expand_dims(cnt_matrix, axis=-1)
cnt_matrix.shape

(214200, 34, 1)

In [25]:
bs, n_total, d_model = cnt_matrix.shape
bs, n_total, d_model

(214200, 34, 1)

In [26]:
n_seq = 33
total_inputs, total_labels = [], []
for i in range(n_total - n_seq):
    x = cnt_matrix[:, i:i + n_seq]
    y = cnt_matrix[:, i + n_seq, :1]
    total_inputs.append(x)
    total_labels.append(y)

In [27]:
if len(total_inputs) == 1:
    train_inputs = total_inputs[0]
else:
    train_inputs = np.concatenate(total_inputs, axis=0)
train_inputs.shape

(214200, 33, 1)

In [28]:
if len(total_labels) == 1:
    train_labels = total_labels[0]
else:
    train_labels = np.concatenate(total_labels, axis=0)
train_labels.shape

(214200, 1)

In [29]:
test_inputs = cnt_matrix[:,1:]
test_inputs.shape

(214200, 33, 1)

In [30]:
def build_model_rnn(n_seq, d_model):
    inputs = tf.keras.layers.Input((n_seq, d_model))  # bs, n_seq, d_model

    hidden = tf.keras.layers.Conv1D(filters=64, kernel_size=6, padding='causal')(inputs)
    hidden = tf.keras.layers.LSTM(units=64, activation=tf.nn.relu)(hidden)  # (bs, units)
    hidden = tf.keras.layers.Dropout(0.4)(hidden)

    output_dense = tf.keras.layers.Dense(1)
    outputs = output_dense(hidden)

    model = tf.keras.Model(inputs=(inputs), outputs=outputs)
    return model

In [31]:
model = build_model_rnn(n_seq, d_model)

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer =tf.keras.optimizers.Adam())
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 33, 1)]           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 33, 64)            448       
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 33,537
Trainable params: 33,537
Non-trainable params: 0
_________________________________________________________________


In [32]:
# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
# save weights
save_weights = tf.keras.callbacks.ModelCheckpoint('weights.hdf5', monitor='loss', verbose=1, save_best_only=True, mode='min', save_freq='epoch', save_weights_only=True)

In [33]:
model.fit(train_inputs, train_labels, epochs = 20, batch_size = 4096, callbacks=[early_stopping, save_weights])

Epoch 1/20
53/53 [==============================] - ETA: 0s - loss: 28.9426
Epoch 00001: loss improved from inf to 28.94257, saving model to weights.hdf5
53/53 [==============================] - 3s 64ms/step - loss: 28.9426
Epoch 2/20
52/53 [============================>.] - ETA: 0s - loss: 29.2474
Epoch 00002: loss did not improve from 28.94257
53/53 [==============================] - 3s 60ms/step - loss: 29.0934
Epoch 3/20
53/53 [==============================] - ETA: 0s - loss: 27.8870
Epoch 00003: loss improved from 28.94257 to 27.88704, saving model to weights.hdf5
53/53 [==============================] - 4s 67ms/step - loss: 27.8870
Epoch 4/20
53/53 [==============================] - ETA: 0s - loss: 27.7062
Epoch 00004: loss improved from 27.88704 to 27.70615, saving model to weights.hdf5
53/53 [==============================] - 3s 63ms/step - loss: 27.7062
Epoch 5/20
52/53 [============================>.] - ETA: 0s - loss: 27.4432
Epoch 00005: loss improved from 27.70615 to 27.3

In [34]:
model = build_model_rnn(n_seq, d_model)
model.load_weights('weights.hdf5')

In [35]:
y_pred = model.predict(test_inputs)
y_pred = np.squeeze(y_pred, axis=-1)
y_pred

array([0.42780167, 0.15667723, 0.3886785 , ..., 0.15080118, 0.15667723,
       0.1518479 ], dtype=float32)

In [36]:
ID = df_test['ID'].values
ID

array([     0,      1,      2, ..., 214197, 214198, 214199])

In [37]:
submission = pd.DataFrame({'ID': ID,'item_cnt_month': y_pred})
submission

,ID,item_cnt_month
0,0,0.427802
1,1,0.156677
2,2,0.388678
3,3,0.217452
4,4,0.156677
...,...,...
214195,214195,0.217597
214196,214196,0.156677
214197,214197,0.150801
214198,214198,0.156677


In [38]:
submission.to_csv('submission.csv',index = False)